In [3]:
import json
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models

import matplotlib.pyplot as plt

import torch

import unicodedata

## 


In [4]:
def carregar_dados(caminho_arquivo_json):
    with open(caminho_arquivo_json, 'r', encoding='utf-8') as arquivo:
        dados = json.load(arquivo)
    return dados


def limpar_texto(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_sem_acento = ''.join([c for c in texto_normalizado if not unicodedata.combining(c)])
    texto_limpo = re.sub(r'[^a-zA-Z0-9\s:;]', '', texto_sem_acento)
    texto_limpo = texto_limpo.lower()
    return texto_limpo

caminho_arquivo_json = "C:/Users/RODRIGO.MAGALHAES/Downloads/Projetos Estudo/Projeto ML/Analise evolucao/tabela_evolucao_nova.json"
dados = carregar_dados(caminho_arquivo_json)

print("Tipo de dados carregados:", type(dados))
print("Exemplo de dados carregados:", dados[:2]) 

dados_df = pd.DataFrame(dados)

dados_df['EVOLUCAO'] = dados_df['EVOLUCAO'].apply(limpar_texto)
dados_df['DS_CID'] = dados_df['DS_CID'].apply(limpar_texto)
dados_df['CIRURGIA'] = dados_df['CIRURGIA'].apply(limpar_texto)

# Substituir strings vazias por NaN
dados_df.replace('', np.nan, inplace=True)

print("Tipo após conversão:", type(dados_df))
print("Estrutura do DataFrame:")

dados_df


Tipo de dados carregados: <class 'list'>
Exemplo de dados carregados: [{'CD_PRE_MED': '876520', 'CD_PACIENTE': '15', 'IDADE_PACIENTE': '16', 'SEXO': 'F', 'DATA_EVOLUCAO': '23/01/20', 'DS_CID': 'NEOPLASIA MALIGNA DO OVÁRIO', 'EVOLUCAO': '# REGISTRO DE CONDUTA MEDICA - PLANTONISTA NAO-ONCOLOGISTA\r\rPACIENTE APRESENTA NOVO EPISODIO DE VOMITO ENEGRECIDO, FLUIDO OU SEM GRUPOS.\rMAE REFERE QUE EPISODIO DE VOMITO FORA PRECIPITADO, EM RAZÃO DE MUDANÇA DE INCLINAÇÃO DE CABECEIRA - DE 45º A 180ºC.\rBRENA INICIOU QUADRO DE DOR ABDOMINAL HA 1 DIA, COM FEZES AMOLECIDAS E ENEGRECIDAS EM 03 EPISODIOS. DEJEÇÕES SE AUSENTARAM POR VOLTA DAS 4H DE 23/01/2020 (MADRUGADA), QUANDO PASSOU A VOMITAR EM GRANDE VOLUME COM COLORAÇÃO SIMILAR A DAS FEZES - TOTAL DE 05 EPISODIOS.\r.\rCIRURGIA AVALIOU E ORIENTOU MANUTENÇÃO DA DIETA ZERO, ALÉM DE PASSAGEM DE SONDA GÁSTRICA  - PACIENTE SERÁ REAVALIADA PELA ESPECIALIDADE EM 24/01/2020.\r.\rSINAIS VITAIS APÓS VOMITO: FR 19 // FC  109 // SPO2 98% // PA 80X58 mmHg.\r.\rA

,CD_PRE_MED,CD_PACIENTE,IDADE_PACIENTE,SEXO,DATA_EVOLUCAO,DS_CID,EVOLUCAO,OBITO,DATA_OBITO,DATA_AVISO_CIRURGIA,CIRURGIA,CLASSIFICACAO_ASA,TIPO_CIRURGIA
0,876520,15,16,F,23/01/20,neoplasia maligna do ovario,registro de conduta medica plantonista naoon...,N,NaN,27/01/20,instalacao de cateter duplo lumen por puncao,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
1,876436,15,16,F,23/01/20,neoplasia maligna do ovario,cirurgia oncologica \r\rcaso:\rataxiatelangec...,N,NaN,27/01/20,instalacao de cateter duplo lumen por puncao,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
2,876520,15,16,F,23/01/20,neoplasia maligna do ovario,registro de conduta medica plantonista naoon...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
3,876436,15,16,F,23/01/20,neoplasia maligna do ovario,cirurgia oncologica \r\rcaso:\rataxiatelangec...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
4,878120,15,16,F,24/01/20,neoplasia maligna do ovario,cipehcb\r pelo plantao vespertino\r\r 7o dpo ...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,180645,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,reavaliacao noturna\r\rmaria cecilia 5 anos ...,N,NaN,27/02/19,laparotomia exploradora,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49996,180645,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,reavaliacao noturna\r\rmaria cecilia 5 anos ...,N,NaN,27/02/19,puncao venosa para acesso venoso central,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49997,180445,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,evolucao vespertina utip hcb\r\r5 anos 14kg\r\...,N,NaN,27/02/19,enteroanastomose qualquer segmento,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49998,180445,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,evolucao vespertina utip hcb\r\r5 anos 14kg\r\...,N,NaN,27/02/19,enterectomia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva


In [5]:
print(dados_df.describe(include='all'))


       CD_PRE_MED CD_PACIENTE IDADE_PACIENTE   SEXO DATA_EVOLUCAO  \
count       50000       50000          50000  50000         50000   
unique      23262         770             27      2          2185   
top        469362       18163             12      M      19/04/19   
freq           20        4782           5967  25041           208   

                              DS_CID  \
count                          50000   
unique                           174   
top     leucemia linfoblastica aguda   
freq                           26855   

                                                 EVOLUCAO  OBITO DATA_OBITO  \
count                                               49996  49732       2387   
unique                                              22993      2         33   
top      evolucao noturna  utip hcb\r\revelyn hilary\r...      N   10/06/24   
freq                                                   40  47345        669   

       DATA_AVISO_CIRURGIA                              C

In [6]:
termos_cancer = [
    'cancer', 'neoplasia', 'tumor', 'metastase', 'oncologia', 'carcinoma',
    'sarcoma', 'linfoma', 'leucemia', 'mieloma', 'melanoma', 'adenocarcinoma',
    'glioma', 'teratoma', 'maligno', 'neoplasia maligna', 'quimioterapia',
    'radioterapia', 'neoadjuvante', 'resseccao', 'biópsia'
]


for termo in termos_cancer:
    count = dados_df['EVOLUCAO'].str.contains(termo, case=False, na=False).sum()
    print(f"Termo '{termo}' aparece {count} vezes.")


Termo 'cancer' aparece 764 vezes.
Termo 'neoplasia' aparece 3152 vezes.
Termo 'tumor' aparece 9295 vezes.
Termo 'metastase' aparece 1252 vezes.
Termo 'oncologia' aparece 5416 vezes.
Termo 'carcinoma' aparece 242 vezes.
Termo 'sarcoma' aparece 2869 vezes.
Termo 'linfoma' aparece 2890 vezes.
Termo 'leucemia' aparece 5811 vezes.
Termo 'mieloma' aparece 0 vezes.
Termo 'melanoma' aparece 5 vezes.
Termo 'adenocarcinoma' aparece 0 vezes.
Termo 'glioma' aparece 1297 vezes.
Termo 'teratoma' aparece 146 vezes.
Termo 'maligno' aparece 91 vezes.
Termo 'neoplasia maligna' aparece 401 vezes.
Termo 'quimioterapia' aparece 7260 vezes.
Termo 'radioterapia' aparece 2549 vezes.
Termo 'neoadjuvante' aparece 15 vezes.
Termo 'resseccao' aparece 4569 vezes.
Termo 'biópsia' aparece 0 vezes.


In [7]:
# Verificar valores ausentes
valores_ausentes = dados_df.isna().sum()
print("Valores ausentes em cada coluna:")
print(valores_ausentes)


Valores ausentes em cada coluna:
CD_PRE_MED                 0
CD_PACIENTE                0
IDADE_PACIENTE             0
SEXO                       0
DATA_EVOLUCAO              0
DS_CID                     0
EVOLUCAO                   4
OBITO                    268
DATA_OBITO             47613
DATA_AVISO_CIRURGIA    13113
CIRURGIA               13113
CLASSIFICACAO_ASA      19761
TIPO_CIRURGIA          16434
dtype: int64


## TNM

In [8]:
df = dados_df.copy()
df

,CD_PRE_MED,CD_PACIENTE,IDADE_PACIENTE,SEXO,DATA_EVOLUCAO,DS_CID,EVOLUCAO,OBITO,DATA_OBITO,DATA_AVISO_CIRURGIA,CIRURGIA,CLASSIFICACAO_ASA,TIPO_CIRURGIA
0,876520,15,16,F,23/01/20,neoplasia maligna do ovario,registro de conduta medica plantonista naoon...,N,NaN,27/01/20,instalacao de cateter duplo lumen por puncao,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
1,876436,15,16,F,23/01/20,neoplasia maligna do ovario,cirurgia oncologica \r\rcaso:\rataxiatelangec...,N,NaN,27/01/20,instalacao de cateter duplo lumen por puncao,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
2,876520,15,16,F,23/01/20,neoplasia maligna do ovario,registro de conduta medica plantonista naoon...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
3,876436,15,16,F,23/01/20,neoplasia maligna do ovario,cirurgia oncologica \r\rcaso:\rataxiatelangec...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
4,878120,15,16,F,24/01/20,neoplasia maligna do ovario,cipehcb\r pelo plantao vespertino\r\r 7o dpo ...,N,NaN,24/01/20,endoscopia digestiva alta com biopsia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,180645,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,reavaliacao noturna\r\rmaria cecilia 5 anos ...,N,NaN,27/02/19,laparotomia exploradora,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49996,180645,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,reavaliacao noturna\r\rmaria cecilia 5 anos ...,N,NaN,27/02/19,puncao venosa para acesso venoso central,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49997,180445,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,evolucao vespertina utip hcb\r\r5 anos 14kg\r\...,N,NaN,27/02/19,enteroanastomose qualquer segmento,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva
49998,180445,24671,5,F,10/01/19,linfoma naohodgkin de tipo nao especificado,evolucao vespertina utip hcb\r\r5 anos 14kg\r\...,N,NaN,27/02/19,enterectomia,ASA III (Paciente com enfermidade sistemica se...,Cirurgia Eletiva


In [9]:
def extrair_trecho_tnm(evolucao):
    if isinstance(evolucao, str):  # Verifica se é uma string
        # Padrão para encontrar menção de T, N, M com números
        padrao = r'\b(t[1-4]|n[0-3]|m[0-1])\b'
        matches = re.finditer(padrao, evolucao.lower())  
        
        trechos = []
        for match in matches:
            pos = match.start()
            trecho_completo = evolucao[max(0, pos-40):min(len(evolucao), pos+len(match.group())+40)]
            trechos.append(trecho_completo.strip())
        
        return trechos if trechos else None  
    
    return None  

In [10]:
df['trecho_tnm'] = df['EVOLUCAO'].apply(extrair_trecho_tnm)


resultados = df[['trecho_tnm', 'CD_PRE_MED']].dropna()
print(resultados)

                                              trecho_tnm CD_PRE_MED
109    [s esfregaco de medula ossea em remissao m1\r\...    1044499
110    [s esfregaco de medula ossea em remissao m1\r\...    1113365
111    [s esfregaco de medula ossea em remissao m1\r\...    1882657
113    [s esfregaco de medula ossea em remissao m1\r\...    3155586
114    [s esfregaco de medula ossea em remissao m1\r\...    3515911
...                                                  ...        ...
49734  [ssificacao siopel pretext 2 c0 e0 f0 h0 m0 n0...     991228
49735  [ssificacao siopel pretext 2 c0 e0 f0 h0 m0 n0...    1011107
49736  [ssificacao siopel pretext 2 c0 e0 f0 h0 m0 n0...    1016849
49737       [ssificacao siopel pretext 2 c0 e0 f0 h0 m0]    1085791
49738  [ssificacao siopel pretext 2 c0 e0 f0 h0 m0 n0...    1096728

[7726 rows x 2 columns]


In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None) 

In [12]:
#restaurar
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth') 

In [13]:
def encontrar_tnm(evolucao):
    if isinstance(evolucao, str):  
        padrao = r'(\b\w+\b)\s(\b\w+\b)\stnm\s(\b\w+\b)\s(\b\w+\b)'
        match = re.search(padrao, evolucao)
        
        if match:
            return match.group(1, 2, 3, 4)
    return None  

In [14]:
df['trecho_tnm'] = df['EVOLUCAO'].apply(encontrar_tnm)

resultados = df[['trecho_tnm', 'CD_PRE_MED']].dropna()
print(resultados)

Empty DataFrame
Columns: [trecho_tnm, CD_PRE_MED]
Index: []


## Normalização dos Dados


In [15]:
# Criar indicadores para presença de cirurgia e data de cirurgia
dados_df['INDICADOR_CIRURGIA'] = dados_df['CIRURGIA'].notna().astype(int)

# Substituir valores conhecidos e tratar valores vazios em OBITO
dados_df['OBITO'] = dados_df['OBITO'].replace({'S': 1, 'N': 0, 'N/A': 0, '': 0})
dados_df['OBITO'] = pd.to_numeric(dados_df['OBITO'], errors='coerce').fillna(0).astype(int)

# Codificação one-hot encoding para a coluna 'SEXO'
dados_df = pd.get_dummies(dados_df, columns=['SEXO'], prefix='', prefix_sep='')

dados_df.rename(columns={'F': 'SEXO_F', 'M': 'SEXO_M'}, inplace=True)

# Converter os valores booleanos True/False para 1/0
dados_df['SEXO_F'] = dados_df['SEXO_F'].astype(int)
dados_df['SEXO_M'] = dados_df['SEXO_M'].astype(int)

C:\Users\RODRIGO.MAGALHAES\AppData\Local\Temp\ipykernel_11116\1928089530.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_df['OBITO'] = dados_df['OBITO'].replace({'S': 1, 'N': 0, 'N/A': 0, '': 0})


In [16]:
print(dados_df.describe(include='all'))

       CD_PRE_MED CD_PACIENTE IDADE_PACIENTE DATA_EVOLUCAO  \
count       50000       50000          50000         50000   
unique      23262         770             27          2185   
top        469362       18163             12      19/04/19   
freq           20        4782           5967           208   
mean          NaN         NaN            NaN           NaN   
std           NaN         NaN            NaN           NaN   
min           NaN         NaN            NaN           NaN   
25%           NaN         NaN            NaN           NaN   
50%           NaN         NaN            NaN           NaN   
75%           NaN         NaN            NaN           NaN   
max           NaN         NaN            NaN           NaN   

                              DS_CID  \
count                          50000   
unique                           174   
top     leucemia linfoblastica aguda   
freq                           26855   
mean                             NaN   
std            

In [17]:
# Convertendo as colunas de datas para o formato de data
dados_df['DATA_EVOLUCAO'] = pd.to_datetime(dados_df['DATA_EVOLUCAO'], errors='coerce')
dados_df['DATA_AVISO_CIRURGIA'] = pd.to_datetime(dados_df['DATA_AVISO_CIRURGIA'], errors='coerce')
dados_df['DATA_OBITO'] = pd.to_datetime(dados_df['DATA_OBITO'], errors='coerce')

# Criar DataFrame para as evoluções
evolucao_df = dados_df[['CD_PRE_MED', 'CD_PACIENTE', 'IDADE_PACIENTE', 'DATA_EVOLUCAO', 'DS_CID', 'EVOLUCAO', 
                        'OBITO', 'SEXO_F', 'SEXO_M']].copy()
evolucao_df['TIPO_EVENTO'] = 'EVOLUCAO'
evolucao_df['DATA_EVENTO'] = evolucao_df['DATA_EVOLUCAO']
evolucao_df = evolucao_df.drop(columns=['DATA_EVOLUCAO'])  # Remove a coluna original de DATA_EVOLUCAO

# Criar DataFrame para as cirurgias
cirurgia_df = dados_df[['CD_PRE_MED', 'CD_PACIENTE', 'IDADE_PACIENTE', 'DATA_AVISO_CIRURGIA', 'DS_CID', 'CIRURGIA', 
                        'CLASSIFICACAO_ASA', 'TIPO_CIRURGIA', 'INDICADOR_CIRURGIA', 'OBITO', 'SEXO_F', 'SEXO_M']].copy()
cirurgia_df['TIPO_EVENTO'] = 'CIRURGIA'
cirurgia_df['DATA_EVENTO'] = cirurgia_df['DATA_AVISO_CIRURGIA']
cirurgia_df = cirurgia_df.drop(columns=['DATA_AVISO_CIRURGIA'])  # Remove a coluna original de DATA_AVISO_CIRURGIA

# Criar DataFrame para os óbitos
obito_df = dados_df[['CD_PRE_MED', 'CD_PACIENTE', 'IDADE_PACIENTE', 'DATA_OBITO', 'DS_CID', 'OBITO', 
                     'SEXO_F', 'SEXO_M']].copy()
obito_df['TIPO_EVENTO'] = 'OBITO'
obito_df['DATA_EVENTO'] = obito_df['DATA_OBITO']
obito_df = obito_df.drop(columns=['DATA_OBITO'])  # Remove a coluna original de DATA_OBITO

# Concatenando os três DataFrames (Evolução, Cirurgia e Óbito) em um só
dados_reestruturados = pd.concat([evolucao_df, cirurgia_df, obito_df], ignore_index=True)

# Excluindo linhas onde DATA_EVENTO é NaT, se for o caso
dados_reestruturados = dados_reestruturados.dropna(subset=['DATA_EVENTO'])

# Reorganizando as colunas para uma melhor visualização
dados_reestruturados = dados_reestruturados[['CD_PRE_MED', 'CD_PACIENTE', 'IDADE_PACIENTE', 'DATA_EVENTO', 'TIPO_EVENTO',
                                             'DS_CID', 'EVOLUCAO', 'CIRURGIA', 'CLASSIFICACAO_ASA', 'TIPO_CIRURGIA', 
                                             'INDICADOR_CIRURGIA', 'OBITO', 'SEXO_F', 'SEXO_M']]



C:\Users\RODRIGO.MAGALHAES\AppData\Local\Temp\ipykernel_11116\3555202482.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dados_df['DATA_EVOLUCAO'] = pd.to_datetime(dados_df['DATA_EVOLUCAO'], errors='coerce')
C:\Users\RODRIGO.MAGALHAES\AppData\Local\Temp\ipykernel_11116\3555202482.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dados_df['DATA_AVISO_CIRURGIA'] = pd.to_datetime(dados_df['DATA_AVISO_CIRURGIA'], errors='coerce')
C:\Users\RODRIGO.MAGALHAES\AppData\Local\Temp\ipykernel_11116\3555202482.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dados_df['DATA_OBIT

In [18]:
dados_reestruturados.sort_values(by=['CD_PACIENTE', 'DATA_EVENTO'], inplace=True)
dados_reestruturados

,CD_PRE_MED,CD_PACIENTE,IDADE_PACIENTE,DATA_EVENTO,TIPO_EVENTO,DS_CID,EVOLUCAO,CIRURGIA,CLASSIFICACAO_ASA,TIPO_CIRURGIA,INDICADOR_CIRURGIA,OBITO,SEXO_F,SEXO_M
19384,215404,10094,6,2019-04-02,EVOLUCAO,neoplasia maligna secundaria de outras partes ...,ambulatorio de oncohematologia pediatrica hcb...,NaN,NaN,NaN,NaN,0,0,1
19385,1237562,10094,7,2020-07-29,EVOLUCAO,neoplasia maligna do tecido conjuntivo e dos t...,29072020 ambulatorio de oncohematologia pedia...,NaN,NaN,NaN,NaN,0,0,1
19386,1627889,10094,8,2021-01-20,EVOLUCAO,neoplasia maligna do encefalo com lesao invasiva,20012021 ambulatorio de oncohematologia pedia...,NaN,NaN,NaN,NaN,0,0,1
19387,2586223,10094,9,2022-02-16,EVOLUCAO,neoplasia maligna do tecido conjuntivo e tecid...,ambulatorio de oncohematologia pediatrica 160...,NaN,NaN,NaN,NaN,0,0,1
19388,2835309,10094,9,2022-05-20,EVOLUCAO,neoplasia maligna do tecido conjuntivo e tecid...,20052022 ambulatorio de oncohematologia pedia...,NaN,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,26081,9964,5,2018-08-27,EVOLUCAO,leucemia mieloide aguda,ambulatorio de oncohematologia pediatrica hos...,NaN,NaN,NaN,NaN,0,1,0
19380,1195069,9964,7,2020-08-07,EVOLUCAO,leucemia linfoblastica aguda,atendimento via fone na data atual,NaN,NaN,NaN,NaN,0,1,0
19381,1600350,9964,7,2021-06-01,EVOLUCAO,leucemia mieloide aguda,ambulatorio de oncohematologia pediatrica \r06...,NaN,NaN,NaN,NaN,0,1,0
19382,3404780,9964,9,2023-06-02,EVOLUCAO,leucemia mieloide aguda,ambulatorio de oncohematologia pediatrica\r \r...,NaN,NaN,NaN,NaN,0,1,0


In [19]:
dados_reestruturados['IDADE_PACIENTE'] = pd.to_numeric(dados_reestruturados['IDADE_PACIENTE'], errors='coerce')
print(dados_reestruturados.dtypes)

CD_PRE_MED                    object
CD_PACIENTE                   object
IDADE_PACIENTE                 int64
DATA_EVENTO           datetime64[ns]
TIPO_EVENTO                   object
DS_CID                        object
EVOLUCAO                      object
CIRURGIA                      object
CLASSIFICACAO_ASA             object
TIPO_CIRURGIA                 object
INDICADOR_CIRURGIA           float64
OBITO                          int32
SEXO_F                         int32
SEXO_M                         int32
dtype: object
